# Importing Libraries

In [2]:
# Linear algebra
import scipy.stats as ss
import scipy.special
from scipy import optimize
from mpmath import gamma
from scipy.stats import expon
import numpy as np
import scipy
from scipy.stats import norm
from numpy import linalg as la
import pandas as pd
from scipy.stats import multivariate_normal
#Plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
### Other
import warnings
warnings.filterwarnings("ignore")
import qmcpy

In [72]:
def NIG_characteristic_function(u, T, r, alpha, beta, delta, DELTA):
  """Compute the characteristic function of the NIG distribution.
  Args:
  - u (array): Vector in R^d.
  - T (float): Terminal time.
  - r (float): Short rate.
  - alpha (float): Alpha parameter of the NIG distribution.
  - beta (array): Array of beta values.
  - delta (float): Delta parameter of the NIG distribution.
  - DELTA (array): Covariance matrix.
  Returns:
  - phi (complex): Characteristic function value.
  """
  mu = -delta * (np.sqrt(alpha**2 - np.square(beta)) - np.sqrt(alpha**2 - np.square(beta + 1)))
  phi = np.exp(np.multiply(1j * T, (r + mu) @ u) + delta * T * (np.sqrt(alpha**2 - beta @ DELTA @ beta) - np.sqrt(alpha**2 - (beta + np.multiply(1j, u)) @ DELTA @ (beta + np.multiply(1j, u)))))
  return phi


def fourier_payoff_call_on_min(u):
  """Compute the Fourier of the payoff of scaled (K = 1) call on min option.
  Args:
  - u (array): Array of Fourier frequencies.
  Returns:
  - payoff (float): Call on min option payoff Fourier transofrm value.
  """
  denominator = (np.multiply(1j, np.sum(u)) - 1) * np.prod(np.multiply(1j, u))
  return 1 / denominator

def fourier_payoff_basket_put(u):
  """Compute the Fourier of the payoff of scaled (K = 1) basket put option.
  Args:
  - u (array): Array of Fourier frequencies.
  Returns:
  - payoff (float): Call on min option payoff Fourier transofrm value.
  """
  numerator = np.prod(scipy.special.gamma(np.multiply(-1j,u)))
  denominator = scipy.special.gamma(-1j*(np.sum(u))+2)
  return (numerator/denominator)


def integrand_to_optimize_NIG_call_on_min(R):
  """Calculate the integrand for QMC estimation of the rainbow option under VG model.
  Args:
  - R (array): Array of damping parameters.
  Returns:
  - integrand (float): Integrand value.
  """
  d = len(S0)  # Dimensionality
  X0 = np.log(np.divide(S0, K))  # Element-wise division
  y = np.multiply(1j, R)
  phi = NIG_characteristic_function(y, T, r, alpha, beta, delta, DELTA)  # Characteristic function
  p = fourier_payoff_call_on_min(y)  # Fourier Transformed Payoff function
  discount = ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-np.dot(R, X0))  # Modified discount factor
  integrand = K * discount * phi * p
  return integrand.real  # Real part of the outcome of Gauss Hermite Quadrature (weighted sum)

def integrand_to_optimize_NIG_basket_put(R):
  """Calculate the integrand for QMC estimation of the rainbow option under VG model.
  Args:
  - R (array): Array of damping parameters.
  Returns:
  - integrand (float): Integrand value.
  """
  d = len(S0)  # Dimensionality
  X0 = np.log(np.divide(S0, d*K))  # Element-wise division
  y = np.multiply(1j, R)
  phi = NIG_characteristic_function(y, T, r, alpha, beta, delta, DELTA)  # Characteristic function
  p = fourier_payoff_basket_put(y)  # Fourier Transformed Payoff function
  discount = ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-np.dot(R, X0))  # Modified discount factor
  integrand = K * discount * phi * p
  return integrand.real  # Real part of the outcome of Gauss Hermite Quadrature (weighted sum)


def exponential_pdf(x, sigma_IS):
  """Compute the probability density function of the exponential distribution.

  Args:
  - x (float): Value to evaluate the PDF at.
  - sigma_IS (float): Scale parameter of the exponential distribution.

  Returns:
  - pdf (float): PDF value at x.
  """
  return np.exp(-np.abs(x) / sigma_IS) / (2 * sigma_IS)

def oneD_exponential_inverse_cdf(u, sigma_IS):
    """Compute the inverse cumulative distribution function of the exponential distribution (1D).

    Args:
    - u (float): Value to evaluate the CDF at.
    - sigma_IS (float): Scale parameter of the exponential distribution.

    Returns:
    - phi_inv (float): Inverse CDF value.
    """
    phi_inv = np.piecewise(u, [0 < u <= 0.5, 1 > u > 0.5], [lambda u: sigma_IS * np.log(2 * u), lambda u: sigma_IS * np.log(1 / (2 - 2 * u))])
    return phi_inv

def exponential_inverse_cdf(u, sigma_IS):
    """Compute the component-wise inverse cumulative distribution function of the exponential distribution (multidimensional) of. avector

    Args:
    - u (array): vector to evaluate the inverse CDF at
    - sigma_IS (float): Scale parameter of the exponential distribution.

    Returns:
    - inv_cdf (array): Inverse CDF vector
    """
    f = np.vectorize(oneD_exponential_inverse_cdf)
    return f(u, sigma_IS)


def RQMC_fourier_NIG_call_on_min_pricer(S0, K, r, T, alpha, beta, delta, DELTA, N, R, m, sigma_IS):
  """Compute the QMC estimate for the rainbow option using exponential IS RQMC under NIG distribution.
  Args:
  - S0 (array): Initial stock prices.
  - K (float): Strike price.
  - r (float): Risk-free interest rate.
  - T (float): Time to maturity.
  - alpha (float): Alpha parameter of the NIG distribution.
  - beta (array): Array of beta values.
  - delta (float): Delta parameter of the NIG distribution.
  - DELTA (array): Covariance matrix.
  - N (int): Number of points per Sobol sequence.
  - R (array): Array of damping parameters.
  - m (int): Number of Sobol sequences.
  - sigma_IS (float): Scale parameter of the IS distribution.
  Returns:
  - qmc_estimate (float): QMC estimate.
  - qmc_stat_error (float): QMC statistical error.
  """
  d = len(S0)  # Number of stocks
  X0 = np.log(np.divide(S0, K))  # Element-wise division
  discount = ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-R @ X0)  # Modified discount factor
  V_list = np.zeros(m)  # Contains m price estimates each corresponding to a shifted sobol sequence
  for i in range(m):
    xi_sobol_scrambled = qmcpy.DigitalNetB2(d, graycode=True, randomize='DS', seed=i).gen_samples(N)
    xi_sobol_shifted_mapped =  np.asarray([exponential_inverse_cdf(row, param) for row, param in zip(zip(*xi_sobol_scrambled), sigma_IS)]).T
    V = 0  # Initialisation of Option price contract value for ith rQMC iteration
    for n in range(N):  # For each Sobol point
      u = xi_sobol_shifted_mapped[n]  # Inverse CDF of uniformly distributed sample to have sample from standard normal
      y = u + np.multiply(1j, R)  # Change of variable, shift by damping parameter to integrate over axis // to real line
      phi = NIG_characteristic_function(y, T, r, alpha, beta, delta, DELTA)  # Evaluation of characteristic function at all Sobol points
      p = fourier_payoff_call_on_min(y)  # Evaluation of Fourier Transformed Payoff function at all Sobol points
      par_vect_t_student_pdf = map(lambda x, args: exponential_pdf(x, args), u, sigma_IS)
      IS_pdf_prod = np.prod(np.asarray(list(par_vect_t_student_pdf)))
      V += (1 / N) * K * discount * np.exp(1j * u @ X0) * phi * p / IS_pdf_prod
    V_list[i] = np.real(V)
  qmc_estimate = np.mean(V_list)  # QMC estimate
  qmc_stat_error = 1.96 * np.std(V_list) / np.sqrt(m)  # QMC statistical error
  return qmc_estimate, qmc_stat_error


def RQMC_fourier_NIG_basket_put_pricer(S0, K, r, T, alpha, beta, delta, DELTA, N, R, m, sigma_IS):
  """Compute the QMC estimate for the basket put option using exponential IS RQMC under NIG distribution.
  Args:
  - S0 (array): Initial stock prices.
  - K (float): Strike price.
  - r (float): Risk-free interest rate.
  - T (float): Time to maturity.
  - alpha (float): Alpha parameter of the NIG distribution.
  - beta (array): Array of beta values.
  - delta (float): Delta parameter of the NIG distribution.
  - DELTA (array): Covariance matrix.
  - N (int): Number of points per Sobol sequence.
  - R (array): Array of damping parameters.
  - m (int): Number of Sobol sequences.
  - sigma_IS (float): Scale parameter of the IS distribution.
  Returns:
  - qmc_estimate (float): QMC estimate.
  - qmc_stat_error (float): QMC statistical error.
  """
  d = len(S0)  # Number of stocks
  X0 = np.log(np.divide(S0, d * K))  # Element-wise division
  discount = ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-R @ X0)  # Modified discount factor
  V_list = np.zeros(m)  # Contains m price estimates each corresponding to a shifted sobol sequence
  for i in range(m):
    xi_sobol_scrambled = qmcpy.DigitalNetB2(d, graycode=True, randomize='DS', seed=i).gen_samples(N)
    xi_sobol_shifted_mapped =  np.asarray([exponential_inverse_cdf(row, param) for row, param in zip(zip(*xi_sobol_scrambled), sigma_IS)]).T
    V = 0  # Initialisation of Option price contract value for ith rQMC iteration
    for n in range(N):  # For each Sobol point
      u = xi_sobol_shifted_mapped[n]  # Inverse CDF of uniformly distributed sample to have sample from standard normal
      y = u + np.multiply(1j, R)  # Change of variable, shift by damping parameter to integrate over axis // to real line
      phi = NIG_characteristic_function(y, T, r, alpha, beta, delta, DELTA)  # Evaluation of characteristic function at all Sobol points
      p = fourier_payoff_basket_put(y)  # Evaluation of Fourier Transformed Payoff function at all Sobol points
      par_vect_t_student_pdf = map(lambda x, args: exponential_pdf(x, args), u, sigma_IS)
      IS_pdf_prod = np.prod(np.asarray(list(par_vect_t_student_pdf)))
      V += (1 / N) * K * discount * np.exp(1j * u @ X0) * phi * p / IS_pdf_prod
    V_list[i] = np.real(V)
  qmc_estimate = np.mean(V_list)  # QMC estimate
  qmc_stat_error = 1.96 * np.std(V_list) / np.sqrt(m)  # QMC statistical error
  return qmc_estimate, qmc_stat_error


def multivariate_laplace_pdf(x, SIGMA_IS, SIGMA_IS_inv):
  """Compute the PDF of the multivariate Laplace distribution.

  Args:
  - x (array): Value to evaluate the PDF at.
  - SIGMA_IS (array): Covariance matrix of the Laplace distribution.
  - SIGMA_IS_inv (array): Inverse of the covariance matrix.

  Returns:
  - f_ML (float): PDF value.
  """
  d = len(x)
  v = (2 - d) / 2
  f_ML = 2 * (2 * np.pi) ** (-d / 2) * (la.det(SIGMA_IS)) ** (-0.5) * (x @ SIGMA_IS_inv @ x / 2) ** (v / 2) * scipy.special.kv(v, np.sqrt(2 * x @ SIGMA_IS_inv @ x))
  return f_ML

def LagRQMC_fourier_NIG_call_on_min_pricer(S0, K, r, T, alpha, beta, delta, DELTA, N, R, M_lag, SIGMA_IS):
  """Compute the QMC estimate using Laguerre quadrature for the rainbow option under NIG distribution.

  Args:
  - S0 (array): Initial stock prices.
  - K (float): Strike price.
  - r (float): Risk-free interest rate.
  - T (float): Time to maturity.
  - alpha (float): Alpha parameter of the NIG distribution.
  - beta (array): Array of beta values.
  - delta (float): Delta parameter of the NIG distribution.
  - DELTA (array): Covariance matrix.
  - N (int): Number of points per Sobol sequence.
  - R (array): Array of damping parameters.
  - M_lag (int): Number of Laguerre quadrature nodes.
  - SIGMA_IS (array): Covariance matrix of the Laplace distribution.
  Returns:
  - V (float): RQMC price estimate.
  """
  d = len(S0)  # Number of stocks
  X0 = np.log(np.divide(S0, K))  # Element-wise division
  discount = ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-R @ X0)  # Modified discount factor
  L_IS = la.cholesky(SIGMA_IS)
  SIGMA_IS_inv = la.inv(SIGMA_IS)
  outer_integral_values = np.zeros(M_lag)  # Store values for the outer integral w.r.t to the Rayleigh distribution
  [lag_abcissas, lag_weights] = np.polynomial.laguerre.laggauss(M_lag)  # Laguerre quadrature nodes and weights
  xi_sobol_scrambled = qmcpy.DigitalNetB2(d, graycode=True, randomize='DS', seed=1).gen_samples(N)
  xi_sobol_mapped = norm.ppf(q=xi_sobol_scrambled, loc=0, scale=1)  # Samples from N(0, I_d)
  for k in range(M_lag):  # Nested integration loop
    inner_integral = 0  # Initialisation of Option price contract value for ith rQMC iteration
    for n in range(N):  # Number of QMC points
      u = L_IS @ xi_sobol_mapped[n] * lag_abcissas[k]  # Sample from N_d(0, SIGMA_IS)
      y = u + np.multiply(1j, R)  # Change of variable, shift by damping parameter
      phi = NIG_characteristic_function(y, T, r, alpha, beta, delta, DELTA)  # Evaluate characteristic function
      p = fourier_payoff_call_on_min(y)  # Evaluate Fourier Transformed Payoff function
      IS_pdf_prod = multivariate_laplace_pdf(u, SIGMA_IS, SIGMA_IS_inv)
      inner_integral_increment = (1 / N) * K * discount * np.real(
          np.exp(1j * u @ X0) * phi * p) / IS_pdf_prod
      if inner_integral_increment - inner_integral_increment != 0: # checking for nan-values due to roundoff error.
          inner_integral_increment = 0
      inner_integral += inner_integral_increment
    outer_integral_values[k] = inner_integral * lag_weights[k] * (
                2 * lag_abcissas[k] * np.exp(-lag_abcissas[k] ** 2)) / np.exp(-lag_abcissas[k])
  V = np.sum(outer_integral_values)
  return V

def LagRQMC_fourier_NIG_basket_put_pricer(S0, K, r, T, alpha, beta, delta, DELTA, N, R, M_lag, SIGMA_IS):
  """Compute the QMC estimate using Laguerre quadrature for the rainbow option under NIG distribution.

  Args:
  - S0 (array): Initial stock prices.
  - K (float): Strike price.
  - r (float): Risk-free interest rate.
  - T (float): Time to maturity.
  - alpha (float): Alpha parameter of the NIG distribution.
  - beta (array): Array of beta values.
  - delta (float): Delta parameter of the NIG distribution.
  - DELTA (array): Covariance matrix.
  - N (int): Number of points per Sobol sequence.
  - R (array): Array of damping parameters.
  - M_lag (int): Number of Laguerre quadrature nodes.
  - SIGMA_IS (array): Covariance matrix of the Laplace distribution.
  Returns:
  - V (float): RQMC price estimate.
  """
  d = len(S0)  # Number of stocks
  X0 = np.log(np.divide(S0, d * K))  # Element-wise division
  discount = ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-R @ X0)  # Modified discount factor
  L_IS = la.cholesky(SIGMA_IS)
  SIGMA_IS_inv = la.inv(SIGMA_IS)
  outer_integral_values = np.zeros(M_lag)  # Store values for the outer integral w.r.t to the Rayleigh distribution
  [lag_abcissas, lag_weights] = np.polynomial.laguerre.laggauss(M_lag)  # Laguerre quadrature nodes and weights
  xi_sobol_scrambled = qmcpy.DigitalNetB2(d, graycode=True, randomize='DS', seed=1).gen_samples(N)
  xi_sobol_mapped = norm.ppf(q=xi_sobol_scrambled, loc=0, scale=1)  # Samples from N(0, I_d)
  for k in range(M_lag):  # Nested integration loop
    inner_integral = 0  # Initialisation of Option price contract value for ith rQMC iteration
    for n in range(N):  # Number of QMC points
      u = L_IS @ xi_sobol_mapped[n] * lag_abcissas[k]  # Sample from N_d(0, SIGMA_IS)
      y = u + np.multiply(1j, R)  # Change of variable, shift by damping parameter
      phi = NIG_characteristic_function(y, T, r, alpha, beta, delta, DELTA)  # Evaluate characteristic function
      p = fourier_payoff_basket_put(y)  # Evaluate Fourier Transformed Payoff function
      IS_pdf_prod = multivariate_laplace_pdf(u, SIGMA_IS, SIGMA_IS_inv)
      inner_integral_increment = (1 / N) * K * discount * np.real(
          np.exp(1j * u @ X0) * phi * p) / IS_pdf_prod
      if inner_integral_increment - inner_integral_increment != 0:
          inner_integral_increment = 0
      inner_integral += inner_integral_increment
    outer_integral_values[k] = inner_integral * lag_weights[k] * (
                2 * lag_abcissas[k] * np.exp(-lag_abcissas[k] ** 2)) / np.exp(-lag_abcissas[k])
  V = np.sum(outer_integral_values)
  return V


def LagRQMC_PCA_fourier_NIG_call_on_min_pricer(S0, K, r, T, alpha, beta, delta, DELTA, N, R, M_lag, SIGMA_IS):
  """Compute the QMC estimate using Laguerre quadrature for the rainbow option under NIG distribution.

  Args:
  - S0 (array): Initial stock prices.
  - K (float): Strike price.
  - r (float): Risk-free interest rate.
  - T (float): Time to maturity.
  - alpha (float): Alpha parameter of the NIG distribution.
  - beta (array): Array of beta values.
  - delta (float): Delta parameter of the NIG distribution.
  - DELTA (array): Covariance matrix.
  - N (int): Number of points per Sobol sequence.
  - R (array): Array of damping parameters.
  - M_lag (int): Number of Laguerre quadrature nodes.
  - SIGMA_IS (array): Covariance matrix of the Laplace distribution.
  Returns:
  - V (float): RQMC price estimate.
  """
  d = len(S0)  # Number of stocks
  X0 = np.log(np.divide(S0, K))  # Element-wise division
  discount = ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-R @ X0)  # Modified discount factor
  eigvals, P = la.eig(SIGMA_IS)
  indices = np.argsort(eigvals)[::-1] # sorting the eigenvalues in descending order from largest to smallest.
  eigvals_sorted = eigvals[indices]
  P_sorted = P[:, indices]
  L_IS = P_sorted @ np.diag(np.sqrt(eigvals_sorted))
  SIGMA_IS_inv = la.inv(SIGMA_IS)
  outer_integral_values = np.zeros(M_lag)  # Store values for the outer integral w.r.t to the Rayleigh distribution
  [lag_abcissas, lag_weights] = np.polynomial.laguerre.laggauss(M_lag)  # Laguerre quadrature nodes and weights
  xi_sobol_scrambled = qmcpy.DigitalNetB2(d, graycode=True, randomize='DS', seed=1).gen_samples(N)
  xi_sobol_mapped = norm.ppf(q=xi_sobol_scrambled, loc=0, scale=1)  # Samples from N(0, I_d)
  for k in range(M_lag):  # Nested integration loop
    inner_integral = 0  # Initialisation of Option price contract value for ith rQMC iteration
    for n in range(N):  # Number of QMC points
      u = L_IS @ xi_sobol_mapped[n] * lag_abcissas[k]  # Sample from N_d(0, SIGMA_IS)
      y = u + np.multiply(1j, R)  # Change of variable, shift by damping parameter
      phi = NIG_characteristic_function(y, T, r, alpha, beta, delta, DELTA)  # Evaluate characteristic function
      p = fourier_payoff_call_on_min(y)  # Evaluate Fourier Transformed Payoff function
      IS_pdf_prod = multivariate_laplace_pdf(u, SIGMA_IS, SIGMA_IS_inv)
      inner_integral_increment = (1 / N) * K * discount * np.real(
          np.exp(1j * u @ X0) * phi * p) / IS_pdf_prod
      if inner_integral_increment - inner_integral_increment != 0:
          inner_integral_increment = 0
      inner_integral += inner_integral_increment
    outer_integral_values[k] = inner_integral * lag_weights[k] * (
                2 * lag_abcissas[k] * np.exp(-lag_abcissas[k] ** 2)) / np.exp(-lag_abcissas[k])
  V = np.sum(outer_integral_values)
  return V

def LagRQMC_PCA_fourier_NIG_basket_put_pricer(S0, K, r, T, alpha, beta, delta, DELTA, N, R, M_lag, SIGMA_IS):
  """Compute the QMC estimate using Laguerre quadrature for the rainbow option under NIG distribution.

  Args:
  - S0 (array): Initial stock prices.
  - K (float): Strike price.
  - r (float): Risk-free interest rate.
  - T (float): Time to maturity.
  - alpha (float): Alpha parameter of the NIG distribution.
  - beta (array): Array of beta values.
  - delta (float): Delta parameter of the NIG distribution.
  - DELTA (array): Covariance matrix.
  - N (int): Number of points per Sobol sequence.
  - R (array): Array of damping parameters.
  - M_lag (int): Number of Laguerre quadrature nodes.
  - SIGMA_IS (array): Covariance matrix of the Laplace distribution.
  Returns:
  - V (float): RQMC price estimate.
  """
  d = len(S0)  # Number of stocks
  X0 = np.log(np.divide(S0, d*K))  # Element-wise division
  discount = ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-R @ X0)  # Modified discount factor
  eigvals, P = la.eig(SIGMA_IS)
  indices = np.argsort(eigvals)[::-1] # sorting the eigenvalues in descending order from largest to smallest.
  eigvals_sorted = eigvals[indices]
  P_sorted = P[:, indices]
  L_IS = P_sorted @ np.diag(np.sqrt(eigvals_sorted))
  SIGMA_IS_inv = la.inv(SIGMA_IS)
  outer_integral_values = np.zeros(M_lag)  # Store values for the outer integral w.r.t to the Rayleigh distribution
  [lag_abcissas, lag_weights] = np.polynomial.laguerre.laggauss(M_lag)  # Laguerre quadrature nodes and weights
  xi_sobol_scrambled = qmcpy.DigitalNetB2(d, graycode=True, randomize='DS', seed=1).gen_samples(N)
  xi_sobol_mapped = norm.ppf(q=xi_sobol_scrambled, loc=0, scale=1)  # Samples from N(0, I_d)
  for k in range(M_lag):  # Nested integration loop
    inner_integral = 0  # Initialisation of Option price contract value for ith rQMC iteration
    for n in range(N):  # Number of QMC points
      u = L_IS @ xi_sobol_mapped[n] * lag_abcissas[k]  # Sample from N_d(0, SIGMA_IS)
      y = u + np.multiply(1j, R)  # Change of variable, shift by damping parameter
      phi = NIG_characteristic_function(y, T, r, alpha, beta, delta, DELTA)  # Evaluate characteristic function
      p = fourier_payoff_basket_put(y)  # Evaluate Fourier Transformed Payoff function
      IS_pdf_prod = multivariate_laplace_pdf(u, SIGMA_IS, SIGMA_IS_inv)
      inner_integral_increment = (1 / N) * K * discount * np.real(
          np.exp(1j * u @ X0) * phi * p) / IS_pdf_prod
      if inner_integral_increment - inner_integral_increment != 0:
          inner_integral_increment = 0
      inner_integral += inner_integral_increment
    outer_integral_values[k] = inner_integral * lag_weights[k] * (
                2 * lag_abcissas[k] * np.exp(-lag_abcissas[k] ** 2)) / np.exp(-lag_abcissas[k])
  V = np.sum(outer_integral_values)
  return V

# Call on min options

## Computing the damping parameters using the rule proposed in [link to the paper](https://arxiv.org/pdf/2203.08196.pdf)

In [44]:
# Payoff parameters
dimension = 5
S0 = 100 * np.ones(dimension)
K = 100
r = 0
T = 1
#NIG Model Parameters
alpha= 12
beta = -3 * np.ones(dimension)
delta= 0.2
DELTA = np.identity(dimension)

############### Setting for the optimal damping parameters #############
# Constraints related to the strip of regularity of the payoff transform
def NIG_constraint(R):
  return alpha**2 - (beta- R) @ DELTA @ (beta - R)

def rainbow_constraint_1(R):
  return -1*R

def rainbow_constraint_2(R):
  return -1 - np.sum(R)

cons = ( {'type': 'ineq', 'fun': NIG_constraint},
        {'type': 'ineq', 'fun': rainbow_constraint_1},
        {'type': 'ineq', 'fun': rainbow_constraint_2},)

# Characteristic function of GBM is an entire function hence there are no related constraints to it.
R_init = -2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_NIG_call_on_min, constraints = cons, x0 = R_init , method = "Trust-Constr" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
print("Optimal damping parameters:", R)

Optimal damping parameters: [-6.15227047 -6.15227047 -6.15227047 -6.15227047 -6.15227048]


## Pricing using RQMC in the Fourier space

### Univariate domain transformation (RQMC)

In [55]:
############### Model and payoff parameters ###############
# Payoff parameters
dimension = 5
S0 = 100 * np.ones(dimension)
K = 100
r = 0
T = 1
#NIG Model Parameters
alpha= 12
beta = -3 * np.ones(dimension)
delta= 0.2
DELTA = np.identity(dimension)

############### QMC parameters ###############
N = 2**6 # number of QMC Sobol points
m = 30 # Number of digital shifts of RQMC
sigma_IS = np.sqrt(2 / (delta**2 * T**2)) * np.ones(dimension)
print("sigma_IS =", sigma_IS)
R_init = -2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_NIG_call_on_min, constraints = cons, x0 = R_init , method = "Nelder-Mead" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
RQMC_estimate, RQMC_stat_error  = RQMC_fourier_NIG_call_on_min_pricer(S0, K, r, T, alpha, beta, delta, DELTA, N, R, m, sigma_IS)
print("RQMC estimate =", round(RQMC_estimate,5), ", Relative Statistical Error =", round(RQMC_stat_error / RQMC_estimate,5)  )

sigma_IS = [7.07106781 7.07106781 7.07106781 7.07106781 7.07106781]
RQMC estimate = 0.08508 , Relative Statistical Error = 0.01074


### Multivariate domain transformation (Laguerre-RQMC)

In [63]:
############### Model and payoff parameters ###############
# Payoff parameters
dimension = 5
S0 = 100 * np.ones(dimension)
K = 100
r = 0
T = 1
#NIG Model Parameters
alpha= 12
beta = -3 * np.ones(dimension)
delta= 0.2
DELTA = np.identity(dimension)

############### QMC parameters ###############
N = 2**6 # number of QMC Sobol points
M_lag = 2**5 # Number of digital shifts of RQMC
SIGMA_IS = (2 / (delta**2 * T**2)) * la.inv(DELTA)
print("SIGMA_IS =", SIGMA_IS)
R_init = -2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_NIG_call_on_min, constraints = cons, x0 = R_init , method = "Nelder-Mead" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
RQMC_estimate = LagRQMC_PCA_fourier_VG_call_on_min_pricer(S0, K, r, T, alpha, beta, delta, DELTA, N, R, M_lag, SIGMA_IS)
print("RQMC estimate =", round(RQMC_estimate,5)  )

SIGMA_IS = [[50.  0.  0.  0.  0.]
 [ 0. 50.  0.  0.  0.]
 [ 0.  0. 50.  0.  0.]
 [ 0.  0.  0. 50.  0.]
 [ 0.  0.  0.  0. 50.]]
RQMC estimate = 0.08835


# Basket put options

## Computing the damping parameters using the rule proposed in [link to the paper](https://arxiv.org/pdf/2203.08196.pdf)

In [64]:
# Payoff parameters
dimension = 5
S0 = 100 * np.ones(dimension)
K = 100
r = 0
T = 1
#NIG Model Parameters
alpha= 12
beta = -3 * np.ones(dimension)
delta= 0.2
DELTA = np.identity(dimension)

############### Setting for the optimal damping parameters #############
# Constraints related to the strip of regularity of the payoff transform
def NIG_constraint(R):
  return alpha**2 - (beta- R) @ DELTA @ (beta - R)

def basket_put_constraint(R):
    return R
cons = ( {'type': 'ineq', 'fun': NIG_constraint},
        {'type': 'ineq', 'fun': basket_put_constraint},)

# Characteristic function of GBM is an entire function hence there are no related constraints to it.
R_init = 2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_NIG_basket_put, constraints = cons, x0 = R_init , method = "Trust-Constr" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
print("Optimal damping parameters:", R)

Optimal damping parameters: [2.16973156 2.16973156 2.16973227 2.16973276 2.16973226]


### Univariate domain transformation (RQMC)

In [74]:
############### Model and payoff parameters ###############
# Payoff parameters
dimension = 2
S0 = 100 * np.ones(dimension)
K = 100
r = 0
T = 1
#NIG Model Parameters
alpha= 12
beta = -3 * np.ones(dimension)
delta= 0.2
DELTA = np.identity(dimension)

############### QMC parameters ###############
N = 2**6 # number of QMC Sobol points
m = 30 # Number of digital shifts of RQMC
sigma_IS = np.sqrt(2 / (delta**2 * T**2)) * np.ones(dimension)
print("sigma_IS =", sigma_IS)
R_init = 2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_NIG_basket_put, constraints = cons, x0 = R_init , method = "Trust-Constr" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
RQMC_estimate, RQMC_stat_error  = RQMC_fourier_NIG_basket_put_pricer(S0, K, r, T, alpha, beta, delta, DELTA, N, R, m, sigma_IS)
print("RQMC estimate =", round(RQMC_estimate,8), ", Relative Statistical Error =", round(RQMC_stat_error / RQMC_estimate,5)  )

sigma_IS = [7.07106781 7.07106781]
RQMC estimate = 3.80287345 , Relative Statistical Error = 0.00466


### Multivariate domain transformation (Laguerre-RQMC)

In [75]:
############### Model and payoff parameters ###############
# Payoff parameters
dimension = 2
S0 = 100 * np.ones(dimension)
K = 100
r = 0
T = 1
#NIG Model Parameters
alpha= 12
beta = -3 * np.ones(dimension)
delta= 0.2
DELTA = np.identity(dimension)

############### QMC parameters ###############
N = 2**6 # number of QMC Sobol points
M_lag = 2**5 # Number of digital shifts of RQMC
SIGMA_IS = (2 / (delta**2 * T**2)) * la.inv(DELTA)
print("SIGMA_IS =", SIGMA_IS)
R_init = 2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_NIG_basket_put, constraints = cons, x0 = R_init , method = "Trust-Constr" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
RQMC_estimate = LagRQMC_PCA_fourier_NIGG_basket_put_pricer(S0, K, r, T, alpha, beta, delta, DELTA, N, R, M_lag, SIGMA_IS)
print("RQMC estimate =", round(RQMC_estimate,5)  )

SIGMA_IS = [[50.  0.]
 [ 0. 50.]]
RQMC estimate = 3.85003
